In [13]:
import gc  
import os  
import time  
import warnings 
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import playground.optivarfuncs as of
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
is_offline = False 
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  
import polars as pl
# set the max columns to none
pd.set_option('display.max_columns', None)

In [20]:
import pandas as pd
import numpy as np
import itertools
def getdfstats(numb_days= 1, stocks=list(range(200)), numb_secs_day= 55):
    '''
    generate a dataframe to hold latest stock near and far prices
    num_days: how many days in dataframe
    stocks: stock ids
    n_nulls: first n rows per day have near and far price = NaN
    drop_days: remove these days from dataframe
    '''
    stockl = np.concatenate(numb_days*[list(itertools.repeat(n, numb_secs_day)) for n in stocks], axis=0).tolist()
    dayl = [np.trunc(d/(numb_secs_day*len(stocks))) for d in list(range(numb_days*numb_secs_day*len(stocks)))]
    vall = [0 for i in list(range(len(stocks)*numb_days*numb_secs_day))]
    secl = [(d % numb_secs_day)*10 for d in list(range(numb_days*numb_secs_day*len(stocks)))]

    df = pd.DataFrame(data={'stock_id': stockl, 'date_id': dayl, 'far_price': vall, 'near_price': vall, 'seconds_in_bucket': secl,'dummynp': vall,'dummyfp': vall})
    return df

# getdfstats()

,stock_id,date_id,far_price,near_price,seconds_in_bucket,dummynp,dummyfp
0,0,0.0,0,0,0,0,0
1,0,0.0,0,0,10,0,0
2,0,0.0,0,0,20,0,0
3,0,0.0,0,0,30,0,0
4,0,0.0,0,0,40,0,0
...,...,...,...,...,...,...,...
10995,199,0.0,0,0,500,0,0
10996,199,0.0,0,0,510,0,0
10997,199,0.0,0,0,520,0,0
10998,199,0.0,0,0,530,0,0


In [21]:
getdfstats(numb_secs_day=1)

,stock_id,date_id,far_price,near_price,seconds_in_bucket,dummynp,dummyfp
0,0,0.0,0,0,0,0,0
1,1,0.0,0,0,0,0,0
2,2,0.0,0,0,0,0,0
3,3,0.0,0,0,0,0,0
4,4,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...
195,195,0.0,0,0,0,0,0
196,196,0.0,0,0,0,0,0
197,197,0.0,0,0,0,0,0
198,198,0.0,0,0,0,0,0


In [ ]:
alldata=pd.read_csv('data/train.csv')

In [42]:
import playground.optivarfuncs as of

def fillrow(row,dfstats):
    #if NaN in row.near_price or row.far_price then replace it with 
    #value in dfstats for the corresponding stock_id and seconds in bucket slot
    #if not NaN then save otherwise save row.near_price and row.far_price in dfstats
    # ex. df.apply(fillrow, axis=1, args=(dfstats,))

    rowstats=(dfstats.stock_id==row.stock_id) &(dfstats.seconds_in_bucket==row.seconds_in_bucket)
    
    if row.near_price==np.NaN:
        row.near_price=dfstats.loc[rowstats,'near_price']
        dfstats.loc[rowstats,'dummynp']=0
    else:
        dfstats.loc[rowstats,'near_price']=row.near_price
        dfstats.loc[rowstats,'dummynp']=1
    
    if row.far_price==np.NaN:
        row.far_price=dfstats.loc[rowstats,'far_price']
        dfstats.loc[rowstats,'dummyfp']=0
    else:
        dfstats.loc[rowstats,'far_price']=row.far_price
        dfstats.loc[rowstats,'dummyfp']=1
    return row
 
  
def interp(df,dfstats):
    #dataframe operated on
    #dfstats - dataframe with stats for each stock
    #max seconds in bucket is 540, once you hit this then recalculate all 54 values
    #for the next day
    if(df.iloc[0,'seconds_in_bucket']==540):
        #for each stock
        for stock in dfstats.stock_id.unique():
            #interpolate near price
            index=dfstats[((dfstats.stock_id==stock)&(dfstats.dummynp==1))].index
            low=dfstats.iloc[index.min()].near_price
            dfstats.iloc[index[0]]=low
            df.interpolate(limit=1, limit_direction='backward')
            dfstats[(dfstats.stock_id==stock)].dummynp=0

            #interpolate far price
            index=dfstats[((dfstats.stock_id==stock)&(dfstats.dummyfp==1))].index
            low=dfstats.iloc[index.min()].near_price
            dfstats.iloc[index[0]]=low
            df.interpolate(limit=1, limit_direction='backward')
            dfstats[(dfstats.stock_id==stock)].dummyfp=0


def estimate(df,dfstats):
    #df - dataframe that needs near_price and far price interpolated
    #dfstats- dataframe with stats for each stock

    df.apply(fillrow, axis=1, args=(dfstats,))
    interp(df,dfstats)
    return df,dfstats


dfstats=getdfstats()

#get the last day of data for all stocks(200stocks*55 seconds_in_bucket per stock)
cache=alldata[-11000:]
_,dfstats=estimate(cache,dfstats)


df=getdfstats(numb_secs_day=1)



estimate(df,dfstats)  


ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [36]:
alldata[-11000:]


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5226980,0,480,0,5372506.50,1,0.999894,18358363.51,NaN,NaN,0.999894,18186.19,1.000070,11974.20,1.000000,-1.279712,26400,480_0_0
5226981,1,480,0,0.00,0,1.000230,1463335.77,NaN,NaN,0.999966,949.20,1.000704,19947.90,1.000000,-4.460216,26400,480_0_1
5226982,2,480,0,2668166.82,-1,1.000514,3178828.19,NaN,NaN,0.999999,194.02,1.000462,63279.86,1.000000,5.899668,26400,480_0_2
5226983,3,480,0,10092140.20,1,0.999887,26401518.01,NaN,NaN,0.999839,41308.00,1.000081,20659.00,1.000000,1.430511,26400,480_0_3
5226984,4,480,0,5413385.50,1,1.000016,14120503.51,NaN,NaN,0.999358,18225.00,1.000071,2006.18,1.000000,-1.220107,26400,480_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [23]:
# test passing a dataframe to a function with apply
def apply_func(row, dfstats):
    # print(dfstats.iloc[0])
    # your code here
    return row
df2=getdfstats(numb_secs_day=1)
df2.apply(fillrow, axis=1, args=(dfstats,))


,stock_id,date_id,far_price,near_price,seconds_in_bucket,dummynp,dummyfp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
195,195.0,0.0,0.0,0.0,0.0,0.0,0.0
196,196.0,0.0,0.0,0.0,0.0,0.0,0.0
197,197.0,0.0,0.0,0.0,0.0,0.0,0.0
198,198.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
from collections import defaultdict 

from dataclasses import dataclass
@dataclass
class Stock:
    #whenever a stock has a non null value add it to this dict 
    #in the form vals[seconds_in_bucket]=price
    near_price_vals=defaultdict(lambda: 0)
    far_price_vals=defaultdict(lambda: 0)

class AllStocks: 
    def __init__(self):
        self.allstocks=defaultdict(lambda: Stock)
    def __getitem__(self,stock_id):
        return self.allstocks[stock_id]
    
seconds_in_bucket=10
stock_id=0
a=AllStocks()
seconds_in_bucket=10

v1=a[stock_id].near_price_vals[seconds_in_bucket]
a[stock_id].near_price_vals[seconds_in_bucket]=100
v1=a[stock_id].near_price_vals[seconds_in_bucket]

In [92]:
# #used for backfilling
# from dataclasses import dataclass

# @dataclass
# class LastValueAllStocks:
#     numberstocks = 200
#     date_id = 0 #initialise to 0
#     all_near_prices = {i:0 for i in range(numberstocks)}
#     all_far_prices = {i:0 for i in range(numberstocks)}

#     def set_initial_prices(self,df):
#         #Set the initial near_and far price for the first entry for each stock in dataframe

#         #first get the very first entry for all stocks
#         earliest_date=df.date_id.min()
#         earliest_seconds_in_bucket=df[df.date_id==earliest_date].seconds_in_bucket.min()
#         indices=df.loc[((df.date_id==earliest_date) & (df.seconds_in_bucket==earliest_seconds_in_bucket)),:].index
 
#         #then loop through and set the near and far prices for each stock in dftmp
#         for index in indices:
#             stk=df[df.index==index].stock_id.values[0]       
#             df.loc[df.index==index,['near_price']]=self.all_near_prices[stk]
#             df.loc[df.index==index,['far_price']]=self.all_far_prices[stk]
#         return df

#     def save_last_prices(self,df):
#         #save the final values for near_and far price for each stock in df

#         #first get the very last entry for all stocks
#         latest_date=df.date_id.max()
#         latest_seconds_in_bucket=df[df.date_id==latest_date].seconds_in_bucket.max()
#         indices=df.loc[((df.date_id==latest_date) & (df.seconds_in_bucket==latest_seconds_in_bucket)),:].index

#         #then loop through and set the near and far prices for each stock in dftmp
#         for index in indices:
#             stk=df[df.index==index].stock_id.values[0] 
#             self.all_near_prices[stk]= df.loc[df.index==index,['near_price']].values[0][0] 
#             self.all_far_prices[stk]= df.loc[df.index==index,['far_price']].values[0][0]
        
# class HandleNaNs:
#     def __init__(self):
#         self.last_value_all_stocks=LastValueAllStocks()

#     def fill_nans(self,df):

#         #sort to get stock ids all together, should be chunks of 200 rows
#         df.sort_values(by=['stock_id','date_id','seconds_in_bucket'],inplace=True)

#         df=self.last_value_all_stocks.set_initial_prices(df)
        
#         #then do a forward interpolation
#         df.far_price=df.far_price.interpolate(method='linear')
#         df.near_price=df.near_price.interpolate(method='linear')

#         self.last_value_all_stocks.save_last_prices(df)
#         df.sort_values(by=['date_id','seconds_in_bucket','stock_id'],inplace=True)
#         return df

In [ ]:
#used for backfilling
from collections import defaultdict 

from dataclasses import dataclass
@dataclass
class Stock:
    #whenever a stock has a non null value add it to this dict 
    #in the form vals[seconds_in_bucket]=price
    near_price_vals={}
    far_price_vals={}

@dataclass
class AllStocks: 
    def __init__(self):
        self.allstocks=defaultdict(lambda: Stock)  


@dataclass
class LastValueAllStocks:
    numberstocks = 200
    date_id = 0 #initialise to 0
    all_near_prices = {i:0 for i in range(numberstocks)}
    all_far_prices = {i:0 for i in range(numberstocks)}

    def set_initial_prices(self,df):
        #Set the initial near_and far price for the first entry for each stock in dataframe

        #first get the very first entry for all stocks
        earliest_date=df.date_id.min()
        earliest_seconds_in_bucket=df[df.date_id==earliest_date].seconds_in_bucket.min()
        indices=df.loc[((df.date_id==earliest_date) & (df.seconds_in_bucket==earliest_seconds_in_bucket)),:].index
 
        #then loop through and set the near and far prices for each stock in dftmp
        for index in indices:
            stk=df[df.index==index].stock_id.values[0]       
            df.loc[df.index==index,['near_price']]=self.all_near_prices[stk]
            df.loc[df.index==index,['far_price']]=self.all_far_prices[stk]
        return df

    def save_last_prices(self,df):
        #save the final values for near_and far price for each stock in df

        #first get the very last entry for all stocks
        latest_date=df.date_id.max()
        latest_seconds_in_bucket=df[df.date_id==latest_date].seconds_in_bucket.max()
        indices=df.loc[((df.date_id==latest_date) & (df.seconds_in_bucket==latest_seconds_in_bucket)),:].index

        #then loop through and set the near and far prices for each stock in dftmp
        for index in indices:
            stk=df[df.index==index].stock_id.values[0] 
            self.all_near_prices[stk]= df.loc[df.index==index,['near_price']].values[0][0] 
            self.all_far_prices[stk]= df.loc[df.index==index,['far_price']].values[0][0]
        
class HandleNaNs:
    def __init__(self):
        self.last_value_all_stocks=LastValueAllStocks()

    def fill_nans(self,df):

        #sort to get stock ids all together, should be chunks of 200 rows
        df.sort_values(by=['stock_id','date_id','seconds_in_bucket'],inplace=True)

        df=self.last_value_all_stocks.set_initial_prices(df)
        
        #then do a forward interpolation
        df.far_price=df.far_price.interpolate(method='linear')
        df.near_price=df.near_price.interpolate(method='linear')

        self.last_value_all_stocks.save_last_prices(df)
        df.sort_values(by=['date_id','seconds_in_bucket','stock_id'],inplace=True)
        return df

In [102]:
import playground.optivarfuncs as of
df1=of.getdf()
df1[-10:]=df1[-10:].fillna(df1.mean())
df1


,stock_id,date_id,far_price,near_price,seconds_in_bucket,target
0,0,0.0,NaN,NaN,0,0
1,0,0.0,NaN,NaN,1,1
2,0,0.0,20.0,21.0,2,2
3,0,0.0,30.0,31.0,3,3
4,1,0.0,NaN,NaN,0,4
5,1,0.0,NaN,NaN,1,5
6,1,0.0,60.0,61.0,2,6
7,1,0.0,70.0,71.0,3,7
8,3,0.0,NaN,NaN,0,8
9,3,0.0,NaN,NaN,1,9


In [ ]:
df2=of.getdf()
df2=hn.fill_nans(df2)
df2

# Now for the real df

In [93]:
df = pd.read_csv("./data/train.csv")
df = df.dropna(subset=["target"])  #drop all rows with NaN in target

In [94]:
hn=HandleNaNs()
df=hn.fill_nans(df)
df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.000000,0.000000,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,0.000000,0.000000,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,0.000000,0.000000,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,0.000000,0.000000,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,0.000000,0.000000,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [132]:
df.stock_id.unique

<bound method Series.unique of 0            0
1            1
2            2
3            3
4            4
          ... 
5237975    195
5237976    196
5237977    197
5237978    198
5237979    199
Name: stock_id, Length: 5237892, dtype: int64>

In [89]:
df.sort_values(by=['date_id','seconds_in_bucket','stock_id'])

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [88]:
df.sort_values(by=['date_id','seconds_in_bucket','stock_id'])

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [61]:
#
df.sort_values(by=['date_id','seconds_in_bucket'])

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [54]:
#get the last row for all stocks (should be 200 rows if no stocks delisted)
latest_date=df.date_id.max()
latest_seconds_in_bucket=df[df.date_id==latest_date].seconds_in_bucket.max()
dftmp=df[(df.date_id==latest_date) & (df.seconds_in_bucket==latest_seconds_in_bucket)]
dftmp
#save the values for near and far price for those stocks




,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5237780,0,480,540,475513.69,-1,0.999193,41686415.27,0.999017,0.999017,0.999193,110123.01,0.999368,283817.38,0.999242,0.760555,26454,480_540_0
5237781,1,480,540,43854.51,-1,0.996543,7680424.49,0.996121,0.996490,0.996543,5675.70,0.997122,167909.10,0.996562,15.859604,26454,480_540_1
5237782,2,480,540,184125.24,-1,0.998246,11081468.02,0.998040,0.998195,0.998143,5616.14,0.998555,102488.46,0.998165,-1.549721,26454,480_540_2
5237783,3,480,540,3635484.09,-1,0.999113,73735938.19,0.998338,0.998386,0.999113,12796.18,0.999209,420250.76,0.999115,3.160238,26454,480_540_3
5237784,4,480,540,2750278.49,-1,0.998700,37418449.82,0.997493,0.998042,0.998645,102169.32,0.998809,583790.75,0.998669,1.009703,26454,480_540_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [64]:
df = pd.read_csv("./data/train.csv")
df = df.dropna(subset=["target"])  #drop all rows with NaN in target
df.sort_values(by=['stock_id','date_id','seconds_in_bucket'],inplace=True)
df[109:141]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
20819,0,1,540,875760.77,1,0.999816,13073132.71,1.001105,1.000353,0.999493,280239.13,0.999816,85596.80,0.999740,-2.700090,109,1_540_0
21010,0,2,0,0.00,0,0.999968,9704431.46,NaN,NaN,0.999968,4055.48,1.000185,23231.88,1.000000,-6.359816,110,2_0_0
21201,0,2,10,0.00,0,0.999751,9705906.23,NaN,NaN,0.999751,4883.95,0.999859,19537.92,0.999772,1.159906,111,2_10_0
21392,0,2,20,0.00,0,0.999534,9705906.23,NaN,NaN,0.999534,26441.31,0.999642,3501.32,0.999630,3.030300,112,2_20_0
21583,0,2,30,0.00,0,0.999425,9705906.23,NaN,NaN,0.999425,10593.80,0.999534,9213.00,0.999483,5.730391,113,2_30_0
21774,0,2,40,0.00,0,0.999317,9808863.44,NaN,NaN,0.999317,36936.11,0.999425,18608.24,0.999389,4.459619,114,2_40_0
21965,0,2,50,0.00,0,0.999317,9911820.65,NaN,NaN,0.999317,22935.39,0.999425,14002.24,0.999384,4.949570,115,2_50_0
22156,0,2,60,0.00,0,0.998774,9911820.65,NaN,NaN,0.998774,56248.66,0.998991,31859.68,0.998913,6.990433,116,2_60_0
22347,0,2,70,0.00,0,0.999317,9911820.65,NaN,NaN,0.999317,46699.77,0.999534,31969.11,0.999446,-1.819730,117,2_70_0
22538,0,2,80,0.00,0,0.999425,10116075.96,NaN,NaN,0.999425,31965.64,0.999642,33538.96,0.999531,-1.670122,118,2_80_0


In [41]:
df1=df.copy()
# dfgb=df.groupby(['stock_id','date_id']).near_price.mean()

#first do a forward interpolation
df1.far_price=df1.far_price.interpolate(method='linear')
df1.near_price=df1.near_price.interpolate(method='linear')
# df1[109:141]

In [97]:
df[-1:].stock_id.values[0]

199

In [131]:
df.far_price.min()

0.0